In [ ]:
Oracle AI Data Platform v1.0

Copyright © 2025, Oracle and/or its affiliates.

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

# OCI GenAI Sentiment Analysis
 **Perform Sentiment Analysis with Llama and GenAI**
 
This notebook demonstrates how to perform sentiment analysis on a data set. You’ll learn:
 
 1. **Creating a dataframe**
 2. **Invoking OCI GenAI service via query_model**
    - Creating an external volume
    - Reading and writing data

 **Prerequisites**

Before you begin, ensure you have:
 - An AI Data Platform instance in a region supporting GenAI.
 - The necessary IAM policies for accessing AI Data Platform. Learn more about permissions.
 - A configured AI Data Platform environment with a compute cluster created.




 **Next Steps**

Follow the step-by-step instructions in this notebook to implement each approach. You can also refer to the following resources for additional guidance:
	•	Working with OCI Gen AI

By the end of this notebook, you’ll be able to experiment with OCI GenAI for doing one of many use cases with OCI GenAI.

## Invoke query_model on Llama 3.3 from Spark

In [ ]:
spark.sql("select query_model('default.oci_ai_models.meta.llama-3.3-70b-instruct','What is Intelligent Data Lake Service in Oracle?') as questions").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|questions |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Perform Sentiment Analysis using GenAI on a sample data set

In [1]:
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql import Row


review_data = [
    "This is a great product!",
    "The service was excellent.",
    "I'm not happy with this purchase.",
    "Highly recommend this place.",
    "Could be better.",
    "Amazing experience!",
    "The food was delicious.",
    "Very disappointing.",
    "A fantastic movie!",
    "Average performance."
]

# 1. Define the schema FIRST
review_schema = StructType([StructField("review", StringType(), True)])

# 2. Create Row objects using the schema
review_row_data = [Row(review=review) for review in review_data]

# 3. NOW create the DataFrame
review_df = spark.createDataFrame(review_row_data, schema=review_schema)  # Schema is essential here.

review_df.show(20, False)

+---------------------------------+
|review |
+---------------------------------+
|This is a great product! |
|The service was excellent. |
|I"m not happy with this purchase.|
|Highly recommend this place. |
|Could be better. |
|Amazing experience! |
|The food was delicious. |
|Very disappointing. |
|A fantastic movie! |
|Average performance. |
+---------------------------------+

In [ ]:
from pyspark.sql.functions import expr
review_df.withColumn("sentiment",\
                     expr("query_model('default.oci_ai_models.meta.llama-3.3-70b-instruct', concat('Answer should return only positive or negative. What is the sentiment for this review: ', review))"))\
.show(10, False)

+---------------------------------+---------+
|review |sentiment|
+---------------------------------+---------+
|This is a great product! |Positive |
|The service was excellent. |Positive.|
|I"m not happy with this purchase.|Negative.|
|Highly recommend this place. |Positive.|
|Could be better. |Negative.|
|Amazing experience! |Positive |
|The food was delicious. |Positive.|
|Very disappointing. |Negative |
|A fantastic movie! |Positive |
|Average performance. |Negative |
+---------------------------------+---------+

In [ ]:
spark.sql("create table default.default.source_data_tab(attributes STRUCT<height: DOUBLE, weight: FLOAT, eye_color: STRING, question: STRING>)").show()
spark.sql("insert into default.default.source_data_tab values(named_struct('height',5.9, 'weight', 180.5, 'eye_color', 'brown', 'question','What is the BMI of this person ?'))")
spark.sql("insert into default.default.source_data_tab values(named_struct('height',5.10, 'weight', 190.5, 'eye_color', 'black', 'question','Is this person fit or overweight ?'))")
spark.sql("insert into default.default.source_data_tab values(named_struct('height',5.11, 'weight', 200.5, 'eye_color', 'turquoise', 'question','What can be the origin country of thie person'))")


++
||
++
++

DataFrame[]

In [ ]:
spark.sql("select *, query_model('default.oci_ai_models.meta.llama-3.3-70b-instruct', attributes) as answer from default.default.source_data_tab").show(10, False)

+-----------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|attributes |answer |
+-----------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
spark.sql("drop table default.default.source_data_tab")


DataFrame[]